# Phase 4v2: CoT 기반 인기글 생성

## 📌 목적
Phase 3v2에서 저장한 파인튜닝 모델을 로드하여,
RPM 팩터 정보와 CoT(Thinking → Response) 구조를 활용한 인기글을 생성합니다.

### 기존 대비 개선점
- ✅ 팩터(인기유형) 정보를 입력으로 제공 → 의도한 스타일 생성
- ✅ CoT: 모델이 먼저 전략을 사고(Thinking) → 그에 맞게 글 생성(Response)
- ✅ EOS 토큰 학습 완료 → 자연스러운 생성 종료

## ⚙️ 사전 조건
- `phase2_dataset_v2.ipynb` 실행으로 `outputs/rpm_factors.json` 존재
- `phase3_finetune_v2.ipynb` 실행으로 `models/llama3_popular_post_lora_v2` 존재
- GPU 런타임 할당

---
## 0. 패키지 설치

In [ ]:
%pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" pandas

print("✅ 패키지 설치 완료")

In [ ]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

print("✅ Google Drive 마운트 완료")

---
## 1. 경로 설정 및 팩터 정보 로드

In [ ]:
from pathlib import Path
import json
import pandas as pd
from IPython.display import display

PROJECT_ROOT = Path("/content/drive/MyDrive/board_crawling")
OUTPUT_DIR   = PROJECT_ROOT / "outputs"
MODEL_DIR    = PROJECT_ROOT / "models"

# ── RPM 팩터 정보 로드 ────────────────────────────────────
FACTORS_PATH = OUTPUT_DIR / "rpm_factors.json"
if FACTORS_PATH.exists():
    with open(FACTORS_PATH, "r", encoding="utf-8") as f:
        factors_data = json.load(f)
    factor_names = [f["factor_name"] for f in factors_data["factors"]]
    factor_stats = factors_data.get("factor_stats", {})
    print(f"✅ 팩터 정보 로드 완료: {len(factor_names)}개 팩터")
    for name in factor_names:
        stats = factor_stats.get(name, {})
        coverage = stats.get("coverage", 0)
        print(f"  → {name} (커버리지: {coverage:.0%})")
else:
    print("⚠️ 팩터 정보 없음, 기본 모드로 생성합니다.")
    factor_names = ["일반"]
    factor_stats = {}

# ── 토픽 키워드 로드 ──────────────────────────────────────
PROMPT_FILES = {
    "익게2":     OUTPUT_DIR / "익게2_topics_for_prompt.json",
    "자유게시판": OUTPUT_DIR / "자유게시판_topics_for_prompt.json",
    "연애상담소": OUTPUT_DIR / "연애상담소_topics_for_prompt.json",
    "익게1":     OUTPUT_DIR / "익게1_topics_for_prompt.json",
}

trend_keywords = {}
for board, path in PROMPT_FILES.items():
    if not path.exists():
        print(f"⚠️ {board}: {path} 없음 → 건너뜀")
        continue
    with open(path, "r", encoding="utf-8") as f:
        topics_data = json.load(f)
    top_topics = sorted(topics_data, key=lambda x: len(x.get("representatives", [])), reverse=True)[:5]
    keywords = []
    for topic in top_topics:
        kw = topic.get("keywords", [])
        if isinstance(kw, list):
            keywords.extend(kw[:8])
    trend_keywords[board] = list(dict.fromkeys(keywords))[:8]
    print(f"✅ {board}: {len(trend_keywords[board])}개 키워드")

if not trend_keywords:
    raise RuntimeError("키워드가 없습니다. Phase 1 노트북을 먼저 실행하세요.")

---
## 2. 파인튜닝 모델 로드

In [ ]:
from unsloth import FastLanguageModel
import torch

FINETUNED_DIR = MODEL_DIR / "llama3_popular_post_lora_v2"
if not FINETUNED_DIR.exists():
    raise FileNotFoundError(f"{FINETUNED_DIR}가 없습니다. Phase 3v2를 먼저 실행하세요.")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=str(FINETUNED_DIR),
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
)

FastLanguageModel.for_inference(model)
print("✅ 파인튜닝 모델 로드 완료")

---
## 3. CoT 기반 생성 함수

In [ ]:
def generate_post_v2(
    board_name: str,
    keywords: list,
    factor_name: str = "일반",
    temperature: float = 0.7,
    max_new_tokens: int = 1024,
) -> dict:
    """
    CoT 기반 인기글 생성.
    모델이 Thinking → Response를 순차 생성하고, 각각 분리하여 반환합니다.
    """
    # 팩터 통계 정보
    stats = factor_stats.get(factor_name, {})
    coverage = stats.get("coverage", 0)
    factor_info = f"{factor_name} (커버리지: {coverage:.0%})" if factor_name != "일반" else "일반"

    instruction = f"주어진 키워드와 인기요인 정보를 사용하여 {board_name}의 인기글 스타일로 글을 작성해줘."
    input_text = f"키워드: {', '.join(keywords)}\n인기요인: {factor_info}"

    BOS = tokenizer.bos_token
    prompt = (
        f"{BOS}Below is an instruction that describes a task. "
        f"Write a response that appropriately completes the request.\n\n"
        f"### Instruction:\n{instruction}\n\n"
        f"### Input:\n{input_text}\n\n"
        f"### Thinking:\n"
    )

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # Llama-3 EOS 토큰 설정
    eos_ids = [tokenizer.eos_token_id]
    eot_id = tokenizer.convert_tokens_to_ids("<|eot_id|>")
    if eot_id != tokenizer.unk_token_id:
        eos_ids.append(eot_id)

    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=eos_ids,
        repetition_penalty=1.2,
        no_repeat_ngram_size=3,
    )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # ── Thinking / Response 분리 ──────────────────────────
    thinking = ""
    response = ""

    if "### Thinking:" in generated_text and "### Response:" in generated_text:
        parts = generated_text.split("### Response:")
        response = parts[-1].strip()

        thinking_part = parts[0]
        if "### Thinking:" in thinking_part:
            thinking = thinking_part.split("### Thinking:")[-1].strip()
    elif "### Response:" in generated_text:
        response = generated_text.split("### Response:")[-1].strip()
    else:
        response = generated_text.strip()

    # ── 제목/본문 분리 ────────────────────────────────────
    lines = response.strip().split("\n")
    if len(lines) > 1:
        title = lines[0].strip()
        content = "\n".join(lines[1:]).strip()
    else:
        title = response[:50].strip() + "..." if len(response) > 50 else response.strip()
        content = response.strip()

    return {
        "board_name": board_name,
        "keywords": ", ".join(keywords),
        "factor": factor_name,
        "thinking": thinking,
        "generated_title": title,
        "generated_content": content,
    }


print("✅ CoT 기반 생성 함수 정의 완료")

---
## 4. 게시판별 + 팩터별 생성 실행

In [ ]:
from datetime import datetime

generated_posts = []

# 각 게시판별로, 상위 2개 팩터에 대해 생성
top_factors = factor_names[:3] if len(factor_names) >= 3 else factor_names

for board, keywords in trend_keywords.items():
    if not keywords:
        continue

    for factor in top_factors:
        print(f"\n🔧 생성 중: [{board}] 팩터={factor}")
        print(f"   키워드: {', '.join(keywords[:5])}...")

        try:
            result = generate_post_v2(
                board_name=board,
                keywords=keywords,
                factor_name=factor,
                temperature=0.7,
                max_new_tokens=1024,
            )
            result["timestamp"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            generated_posts.append(result)

            print(f"   ✅ 생성 완료")
            print(f"   💭 Thinking: {result['thinking'][:100]}...")
            print(f"   📝 제목: {result['generated_title']}")
            print(f"   📄 내용: {result['generated_content'][:150]}...")

        except Exception as exc:
            print(f"   ⚠️ 생성 실패: {exc}")

---
## 5. 생성 결과 저장 및 비교

In [ ]:
if generated_posts:
    result_df = pd.DataFrame(generated_posts)
    output_csv = OUTPUT_DIR / "generated_posts_v2.csv"
    result_df.to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"\n✅ 생성 결과 저장 완료: {output_csv}")

    print("\n" + "=" * 70)
    print("생성 결과 요약".center(70))
    print("=" * 70)
    display(result_df[["board_name", "factor", "generated_title"]].head(20))
else:
    print("⚠️ 생성된 게시글이 없습니다.")

In [ ]:
# ── 상세 출력: Thinking + Response ────────────────────────
print("\n" + "=" * 70)
print("CoT 상세 출력 (Thinking → Response)".center(70))
print("=" * 70)

for i, post in enumerate(generated_posts[:5], 1):
    print(f"\n[{i}] [{post['board_name']}] 팩터: {post['factor']}")
    print(f"    키워드: {post['keywords']}")
    print(f"\n    💭 Thinking:")
    print(f"    {post['thinking'][:200]}")
    print(f"\n    📝 Response:")
    print(f"    제목: {post['generated_title']}")
    print(f"    내용: {post['generated_content'][:300]}")
    print("-" * 70)

---
## 6. 추가 팁
- 생성 결과를 수작업으로 검토하여 품질 확인 후 배포하세요.
- `temperature`, `factor_name`, 키워드 조합을 조정하여 다양한 샘플을 생성할 수 있습니다.
- 특정 팩터를 지정하면 해당 인기유형에 맞는 글이 생성됩니다.
- CoT의 Thinking 부분이 생성 전략을 보여주므로, 모델의 추론 과정을 해석할 수 있습니다.